In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colrs
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, utils, datasets, models
import sys 
sys.path.append("..") 
from load_data_task3_aug import DataGenerator
import cv2
import copy 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import os  
# os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

input_path =  "/ICASSP2025_Dataset/Inputs/Task_3_ICASSP/" 
output_path = "/ICASSP2025_Dataset/Outputs/Task_3_ICASSP/" 
csv_path = "/ICASSP2025_Dataset/"

Input_list_file_names_train = []
for b in range(1, 26):  # 25 for train 1 for eval 
    for ant in range(1, 2):
        if ant == 1:
            s_range = range(50)
        else:
            s_range = range(80)
        for s in s_range:
            for f in {1, 2, 3}:
                Input_list_file_names_train.append("B" + str(b) + "_Ant" + str(ant) + "_f" + str(f) + "_S" + str(s))

Input_list_file_names_eval = []
for b in range(25, 26):  # 25 for train 1 for eval 
    for ant in range(1, 2):
        if ant == 1:
            s_range = range(50)
        else:
            s_range = range(80)
        for s in s_range:
            for f in {1, 2, 3}:
                Input_list_file_names_eval.append("B" + str(b) + "_Ant" + str(ant) + "_f" + str(f) + "_S" + str(s))



Input_list_IDs_train =  np.arange(0, len(Input_list_file_names_train), 1, dtype=int)

train_generator = DataGenerator (device, Input_list_IDs_train, Input_list_file_names_train, input_path, output_path,
                 dim_X = (256,256), dim_y = (256,256),  csv_path=csv_path, random_augment = 1,
                 shuffle=True)

Input_list_IDs_eval =  np.arange(0, len(Input_list_file_names_eval), 1, dtype=int)
eval_generator = DataGenerator (device, Input_list_IDs_eval, Input_list_file_names_eval, input_path, output_path,
                 dim_X = (256,256), dim_y = (256,256),  csv_path=csv_path, random_augment = 1,
                 shuffle=True)


dataloaders = {
    'train': DataLoader(train_generator, batch_size=16, shuffle=True),
    'val': DataLoader(eval_generator, batch_size=16, shuffle=True)
}
print(len(train_generator), len(eval_generator), device)

3750 150 cuda


In [ ]:
# # Try the best aug method based on kaggle result
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# input_path =  "/ICASSP2025_Dataset/Inputs/Task_3_ICASSP/" 
# output_path = "/ICASSP2025_Dataset/Outputs/Task_3_ICASSP/" 
# csv_path = "/ICASSP2025_Dataset/"

# Input_list_file_names_train = []
# for b in range(1, 26):  # 25 for train 1 for eval 
#     for ant in range(1, 2):
#         if ant == 1:
#             s_range = range(50)
#         else:
#             s_range = range(80)
#         for s in s_range:
#             for f in {1, 2, 3}:
#                 Input_list_file_names_train.append("B" + str(b) + "_Ant" + str(ant) + "_f" + str(f) + "_S" + str(s))

# Input_list_file_names_eval = []
# for b in range(25, 26):  # 25 for train 1 for eval 
#     for ant in range(1, 2):
#         if ant == 1:
#             s_range = range(50)
#         else:
#             s_range = range(80)
#         for s in s_range:
#             for f in {1, 2, 3}:
#                 Input_list_file_names_eval.append("B" + str(b) + "_Ant" + str(ant) + "_f" + str(f) + "_S" + str(s))

# augment_modes = ["original", "flip_y", "flip_x", "rotate_90", "rotate_180", "rotate_270", "flip_x_rotate_90", "flip_y_rotate_90"]

# Input_list_IDs_train =  np.arange(0, len(Input_list_file_names_train), 1, dtype=int)
# Input_list_IDs_eval =  np.arange(0, len(Input_list_file_names_eval), 1, dtype=int)

# train_dataset = ConcatDataset([
#     DataGenerator(device, Input_list_IDs_train, Input_list_file_names_train, input_path, output_path, (256, 256), (256, 256), csv_path, random_augment=0,
#                   shuffle=True, augment_mode=mode)
#     for mode in augment_modes
# ])
# eval_dataset = ConcatDataset([
#     DataGenerator(device, Input_list_IDs_eval, Input_list_file_names_eval, input_path, output_path, (256, 256), (256, 256), csv_path, random_augment=0,
#                   shuffle=True, augment_mode=mode)
#     for mode in augment_modes
# ])

# dataloaders = {
#     'train': DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True),
#     'val': DataLoader(eval_dataset, batch_size=16, shuffle=True, drop_last=True)
# }
# print(len(train_dataset), len(eval_dataset), device)

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from collections import defaultdict
import torch.nn.functional as F
from torchmetrics import StructuralSimilarityIndexMeasure as SSIM
from torchmetrics.functional.image import image_gradients
import torch.nn as nn
from tqdm import tqdm 

def calc_loss_mix_ssim_gradient(pred, target, metrics):
    target = target.unsqueeze(1)
    target_dy, target_dx = image_gradients(target)
    pred_dy, pred_dx = image_gradients(pred)
    gdl = (target_dy - pred_dy).pow(2) + (target_dx - pred_dx).pow(2)
    loss1 = gdl.sum() / pred.numel()

    ssim = SSIM(data_range=255.0).to(device)
    loss2 = 1 - ssim(pred, target)

    l1_loss = nn.L1Loss()
    loss_l1 = l1_loss(pred, target)

    mse = nn.MSELoss()
    loss_mse = mse(pred, target)

    loss = 100 * loss2 + loss_l1
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    metrics['L1 loss'] += loss_l1.data.cpu().numpy() * target.size(0)
    metrics['1-ssim loss'] += loss2.data.cpu().numpy() * target.size(0)
    metrics['GDL loss'] += loss1.data.cpu().numpy() * target.size(0)
    metrics['MSE loss'] += loss_mse.data.cpu().numpy() * target.size(0)

    return loss

def calc_discriminator_loss(real_output, fake_output, metrics):
    criterion = nn.BCEWithLogitsLoss()
    real_labels = torch.ones(real_output.size(0), 1).to(device)
    fake_labels = torch.zeros(fake_output.size(0), 1).to(device)
    
    real_loss = criterion(real_output, real_labels)
    fake_loss = criterion(fake_output, fake_labels) 

    return real_loss + fake_loss

def print_metrics(metrics, epoch_samples, phase):
    outputs1 = []
    for k in metrics.keys():
        outputs1.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
    print("{}: {}".format(phase, ", ".join(outputs1)))

def train_gan(generator, discriminator, g_optimizer, d_optimizer, device, num_epochs=10):
    best_model_wts = copy.deepcopy(generator.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                generator.train() 
                discriminator.train()  
            else:
                generator.eval()   
                discriminator.eval()  

            metrics = defaultdict(float)
            epoch_samples = 0

            with tqdm(dataloaders[phase], unit="batch", disable=False) as tepoch:
                for inputs, targets in tepoch:
                    tepoch.set_description(f"Epoch {epoch + 1}/{num_epochs} ({phase})")

                    inputs = inputs.permute(0, 3, 1, 2)  # bx3x256x256
                    inputs = inputs.float().to(device)
                    targets = targets.float().to(device)

                    d_optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase == 'train'):
                        fake_images = generator(inputs).detach() 
                        
                        combined_targets = torch.cat((inputs, targets.unsqueeze(1)), dim=1)
                        combined_fake = torch.cat((inputs, fake_images), dim=1)

                        real_output = discriminator(combined_targets)
                        fake_output = discriminator(combined_fake)

                        d_loss = calc_discriminator_loss(real_output, fake_output, metrics)
                        if phase == 'train':
                            d_loss.backward()
                            d_optimizer.step()
                    
                    g_optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase == 'train'):
                        fake_images = generator(inputs)
                        combined_fake = torch.cat((inputs, fake_images), dim=1)
                        fake_output = discriminator(combined_fake)
                        
                        # g_loss = -torch.mean(fake_output) + calc_loss_mix_ssim_gradient(fake_images, targets, metrics) # find best kaggle result
                        g_loss = calc_loss_mix_ssim_gradient(fake_images, targets, metrics)
                        
                        if phase == 'train':
                            g_loss.backward()
                            g_optimizer.step()

                    epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            if phase == 'val':
                # model_save_path = f"/data1/lihan_data/task3/20241120_ft/t3model_epoch{epoch + 1}_valloss{epoch_loss:.4f}.pt"
                # torch.save(generator.state_dict(), model_save_path)
                # print(f"Model saved at {model_save_path}")
                
                if epoch_loss < best_loss:
                    print("saving best model")
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(generator.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))
    
    generator.load_state_dict(best_model_wts)
    return generator, discriminator

In [ ]:
from model.SIP2Net import SIP2Net

model_G = SIP2Net(
    n_blocks=[3, 3, 9, 3],  
    atrous_rates=[2, 4, 6],  
    multi_grids=[1, 2, 4],  
    output_stride=8,        
)
model_G.to(device)

from model.dis_model import Discriminator
model_D = Discriminator()
model_D.to(device)

# %%
import torch
import torch.optim as optim
# from torch.optim import lr_scheduler
import time
import copy

torch.backends.cudnn.enabled

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

optimizer_G = optim.AdamW(model_G.parameters(), lr=0.00005, weight_decay=1e-5)
optimizer_D = optim.AdamW(model_D.parameters(), lr=0.00005, weight_decay=1e-5)

model = train_gan(generator=model_G, discriminator=model_D, g_optimizer=optimizer_G, d_optimizer=optimizer_D, device=device, num_epochs=50)




cuda:0
Epoch 0/49
----------


Epoch 1/50 (train):   1%|          | 2/235 [00:07<14:52,  3.83s/batch]